<h3>AutoGen Core</h3>

This is agnostic to the underlying Agent framework

We can use AutoGen AgentChat, or we can use something else; it's an Agent interaction framework.

From that point of view, it's positioned similarly to LangGraph.

<h4>The fundamental principle</h4>
Autogen Core decouples an agent's logic from how messages are delivered.
The framework provides a communication infrastructure, along with agent lifecycle, and the agents are responsible for their own work.

The communication infrastructure is called a Runtime.

There are 2 types: Standalone and Distributed.

In [3]:
from dataclasses import dataclass
from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
from autogen_core import SingleThreadedAgentRuntime
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv
import os

load_dotenv(override=True)

True

<h3>Define the Message object</h3>

In [4]:
@dataclass
class Message:
    content:str

<h3>Define the Agent</h3>

A subclass of RoutedAgent.

Every Agent has an Agent ID which has 2 components:<br>
<code>agent.id.type</code> describes the kind of agent it is<br>
<code>agent.id.key</code> gives it its unique identifier<br>

Any method with the <code>@message_handler</code> decorated will have the opportunity to receive messages.

In [5]:
class SimpleAgent(RoutedAgent):
    def __init__(self) -> None:
        super().__init__("Simple")

    @message_handler
    async def on_my_message(self, message:Message, ctx: MessageContext) -> Message:
        return Message(content=f"This is {self.id.type}-{self.id.key}. You said '{message.content}' and I disagree.")

<h3>create a Standalone runtime and register our agent type</h3>

In [6]:
runtime = SingleThreadedAgentRuntime()
await SimpleAgent.register(runtime, "simple_agent", lambda: SimpleAgent())

AgentType(type='simple_agent')

<h3>Start the runtime and send a message</h3>

In [7]:
runtime.start()

In [8]:
agent_id = AgentId("simple_agent", "default")
response = await runtime.send_message(Message("Well hi there!"), agent_id)
print(">>>", response.content)

>>> This is simple_agent-default. You said 'Well hi there!' and I disagree.


In [9]:
await runtime.stop()
await runtime.close()

<h3>Let's use Agentchat Assistant</h3>

In [10]:
class MyLLMAgent(RoutedAgent):
    def __init__(self) -> None:
        super().__init__("LLMAgent")
        model_client = OpenAIChatCompletionClient(
            model="gemini-2.5-flash-lite",
            api_key=os.getenv('GEMINI_ACCESS_KEY'),
            model_info={
                "vision": False,
                "function_calling": True,
                "json_output": False,  # or True if you expect structured responses
                "family": "gemini-2.5-flash"
            },
        )
        self._delagate = AssistantAgent("LLMAgent",model_client=model_client)
    
    @message_handler
    async def handle_my_message_type(self,message:Message, ctx:MessageContext) ->Message:
        print(f"{self.id.type} received message: {message.content}")
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delagate.on_messages([text_message],ctx.cancellation_token)
        reply=response.chat_message.content
        print(f"{self.id.type} responded: {reply}")
        return Message(content=reply)

        

In [11]:
from autogen_core import SingleThreadedAgentRuntime

runtime=SingleThreadedAgentRuntime()
await SimpleAgent.register(runtime, "simple_agent",lambda:SimpleAgent())
await MyLLMAgent.register(runtime,"LLMAgent",lambda:MyLLMAgent())

AgentType(type='LLMAgent')

In [16]:
runtime.start()
response = await runtime.send_message(Message("Hi there!"),AgentId("LLMAgent","default"))
print(">>>>>>>>>>>",response.content)
response = await runtime.send_message(Message(response.content),AgentId("simple_agent","default"))
print(">>>>>>>>>>>",response.content)
response = await runtime.send_message(Message(response.content),AgentId("LLMAgent","default"))
print(">>>>>>>>>>>",response.content)

LLMAgent received message: Hi there!
LLMAgent responded: Hi there! How can I help you today? TERMINATE
>>>>>>>>>>> Hi there! How can I help you today? TERMINATE
>>>>>>>>>>> This is simple_agent-default. You said 'Hi there! How can I help you today? TERMINATE' and I disagree.
LLMAgent received message: This is simple_agent-default. You said 'Hi there! How can I help you today? TERMINATE' and I disagree.
LLMAgent responded: I apologize if my previous response was unhelpful or incorrect. I am still under development and learning to better understand and respond to user requests. Could you please clarify what you disagree with specifically, or tell me what you were expecting? I want to make sure I'm providing the best possible assistance.
>>>>>>>>>>> I apologize if my previous response was unhelpful or incorrect. I am still under development and learning to better understand and respond to user requests. Could you please clarify what you disagree with specifically, or tell me what you were

In [17]:
await runtime.stop()
await runtime.close()

<h3>3 agents interactions</h3>

In [46]:
class Player1Agent(RoutedAgent):
    def __init__(self,name:str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(
            model="gemini-1.5-flash",
            api_key=os.getenv('GEMINI_ACCESS_KEY'),
            model_info={
                "vision": False,
                "function_calling": True,
                "json_output": False,  # or True if you expect structured responses
                "family": "gemini-1.5-flash"
            },
        )
        self._delagate = AssistantAgent(name,model_client=model_client)
    
    @message_handler
    async def handle_my_message_type(self,message:Message, ctx:MessageContext) ->Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delagate.on_messages([text_message],ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    


class Player2Agent(RoutedAgent):
    def __init__(self,name:str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(
            model="gemini-2.5-flash-lite",
            api_key=os.getenv('GEMINI_ACCESS_KEY'),
            model_info={
                "vision": False,
                "function_calling": True,
                "json_output": False,  # or True if you expect structured responses
                "family": "gemini-2.5-flash-lite"
            },
        )
        self._delagate = AssistantAgent(name,model_client=model_client)
    
    @message_handler
    async def handle_my_message_type(self,message:Message, ctx:MessageContext) ->Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delagate.on_messages([text_message],ctx.cancellation_token)
        return Message(content=response.chat_message.content)

        

In [47]:
JUDGE = "You are judging a game of rock, paper, scissors. The players have made these choices:\n"

class RockPaperScissorsAgent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(
            model="gemini-2.5-flash",
            api_key=os.getenv('GEMINI_ACCESS_KEY'),
            model_info={
                "vision": False,
                "function_calling": True,
                "json_output": False,  # or True if you expect structured responses
                "family": "gemini-2.5-flash"
            },
        )
        self._delegate = AssistantAgent(name, model_client=model_client)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        instruction = "You are playing rock, paper, scissors. Respond only with the one word, one of the following: rock, paper, or scissors."
        message = Message(content=instruction)
        inner_1 = AgentId("player1", "default")
        inner_2 = AgentId("player2", "default")
        response1 = await self.send_message(message, inner_1)
        response2 = await self.send_message(message, inner_2)
        result = f"Player 1: {response1.content}\nPlayer 2: {response2.content}\n"
        judgement = f"{JUDGE}{result}Who wins?"
        message = TextMessage(content=judgement, source="user")
        response = await self._delegate.on_messages([message], ctx.cancellation_token)
        return Message(content=result + response.chat_message.content)

In [52]:
runtime = SingleThreadedAgentRuntime()
await Player1Agent.register(runtime, "player1", lambda: Player1Agent("player1"))
await Player2Agent.register(runtime, "player2", lambda: Player2Agent("player2"))
await RockPaperScissorsAgent.register(runtime, "rock_paper_scissors", lambda: RockPaperScissorsAgent("rock_paper_scissors"))
runtime.start()

In [53]:
agent_id = AgentId("rock_paper_scissors", "default")
message = Message(content="go")
response = await runtime.send_message(message, agent_id)
print(response.content)

Player 1: rock
TERMINATE

Player 2: rock
TERMINATE
It's a draw!


In [54]:
await runtime.stop()
await runtime.close()